# Проект по SQL

<div style="border:solid blue 2px; padding: 20px">

**Описание проекта:**
Во время коронавируса у людей стало больше времени для книг. Компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Цель проекта:** Проанализировать базу данных

**Задания:**
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных:**

Таблица `books`

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`

Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## Изучение данных

### Таблица `books`

In [2]:
# Посмотрим на таблицу books
query = '''
            SELECT * FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# Выведем информацию о данных
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [4]:
# Посчитаем количество пропусков
books.isna().sum()

book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64

In [5]:
# Посчитаем дубликаты
books.duplicated().sum()

0

In [6]:
# Посчитаем дубликаты по названию и автору книги
books[books.duplicated(subset = ['title', 'author_id'], keep = False)]

,book_id,author_id,title,num_pages,publication_date,publisher_id
425,426,39,Memoirs of a Geisha,434,2005-11-15,241
426,427,39,Memoirs of a Geisha,503,2005-11-22,311


В таблице `books` 1000 записей, явных дубликатов и пропусков нет, неявные дубликаты есть, но это две разные версии книги.

### Таблица `authors`

In [7]:
# Посмотрим на таблицу authors
query = '''
            SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
# Выведем информацию о данных
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [9]:
# Посчитаем количество пропусков
authors.isna().sum()

author_id    0
author       0
dtype: int64

In [10]:
# Посчитаем дубликаты
authors.duplicated().sum()

0

Таблица `authors` содержит 636 записей, дубликатов и пропусков не обнаружено.

### Таблица `publishers`

In [11]:
# Посмотрим на таблицу authors
query = '''
            SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [12]:
# Выведем информацию о данных
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [13]:
# Посчитаем количество пропусков
publishers.isna().sum()

publisher_id    0
publisher       0
dtype: int64

In [14]:
# Посчитаем дубликаты
publishers.duplicated().sum()

0

Таблица `publishers` содержит 340 записей, дубликатов и пропусков не обнаружено.

### Таблица `ratings`

In [15]:
# Посмотрим на таблицу ratings
query = '''
            SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [16]:
# Выведем информацию о данных
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [17]:
# Посчитаем количество пропусков
ratings.isna().sum()

rating_id    0
book_id      0
username     0
rating       0
dtype: int64

In [18]:
# Посчитаем дубликаты
ratings.duplicated().sum()

0

Таблица `ratings` содержит 6456 записей, дубликатов и пропусков не обнаружено.

### Таблица `reviews`

In [19]:
# Посмотрим на таблицу reviews
query = '''
            SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [20]:
# Выведем информацию о данных
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [21]:
# Посчитаем количество пропусков
reviews.isna().sum()

review_id    0
book_id      0
username     0
text         0
dtype: int64

In [22]:
# Посчитаем дубликаты
reviews.duplicated().sum()

0

Таблица `reviews` содержит 2793 записей, дубликатов и пропусков не обнаружено.

## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [23]:
query = '''
            SELECT COUNT(book_id)
            FROM books
            WHERE CAST(publication_date AS timestamp) > '2000-01-01'
        '''
books_after_millennium = pd.io.sql.read_sql(query, con = engine)
books_after_millennium

,count
0,819


*После 1 января 2000 года вышло 819 книг.*

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [24]:
query = '''
            WITH 
                 rv AS
                       (SELECT rev.book_id,
                               CAST(COUNT(rev.review_id) AS integer) AS count_reviews
                        FROM reviews AS rev
                        GROUP BY rev.book_id),
                 rt AS 
                       (SELECT rat.book_id,
                               AVG(rat.rating) AS avg_rating
                        FROM ratings AS rat
                        GROUP BY rat.book_id)
                           
            SELECT b.title,
                   b.book_id,
                   rv.count_reviews,
                   rt.avg_rating
            FROM books AS b
            JOIN rv on b.book_id=rv.book_id
            LEFT JOIN rt on b.book_id=rt.book_id
            ORDER BY rv.count_reviews DESC
        '''
books_rating = pd.io.sql.read_sql(query, con = engine)
books_rating

,title,book_id,count_reviews,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,The Book Thief,656,6,4.264151
2,Outlander (Outlander #1),497,6,4.125000
3,Water for Elephants,963,6,3.977273
4,The Da Vinci Code (Robert Langdon #2),696,6,3.830508
...,...,...,...,...
989,Stone of Farewell (Memory Sorrow and Thorn #2),599,1,5.000000
990,Debt of Honor (Jack Ryan #7),187,1,3.000000
991,Color: A Natural History of the Palette,156,1,4.333333
992,Winter Prey (Lucas Davenport #5),984,1,4.500000


Лидер по количеству обзоров "Сумерки", но оценка при этом средняя. 

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [25]:
query = '''
            WITH bk AS 
                       (SELECT b.publisher_id,
                               COUNT(b.book_id) AS book_count
                        FROM books AS b
                        WHERE num_pages > 50
                        GROUP BY b.publisher_id)
                           
            SELECT p.publisher,
                   bk.book_count
            FROM publishers AS p
            JOIN bk on p.publisher_id=bk.publisher_id
            ORDER BY bk.book_count DESC
        '''
not_brochure_publisher = pd.io.sql.read_sql(query, con = engine)
not_brochure_publisher.head()

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


Больше всего книг толще 50 страниц у издательств Penguin Books, Vintage и Grand Central Publishing.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [26]:
query = '''
           WITH rt AS 
                      (SELECT book_id
                      FROM ratings
                      GROUP BY book_id
                      HAVING COUNT(rating_id) >= 50)
                      
            SELECT a.author,
                   AVG(r.rating) AS avg_raiting
            FROM books AS b
            JOIN authors AS a ON a.author_id = b.author_id
            JOIN ratings AS r ON r.book_id = b.book_id
            JOIN rt ON rt.book_id = b.book_id
            GROUP BY a.author
            ORDER BY avg_raiting DESC
            LIMIT 1
        '''
top_author = pd.io.sql.read_sql(query, con = engine)
top_author

,author,avg_raiting
0,J.K. Rowling/Mary GrandPré,4.287097


Лидеры по оценке и количеству оценок и средней оценке соавторы книг о Гарри Поттере Дж. К. Роулинг и Мари ГранПре.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [27]:
query = '''
            WITH r AS 
                      (SELECT COUNT(review_id) AS count_reviews
                       FROM reviews
                       WHERE username IN 
                                         (SELECT username
                                         FROM ratings
                                         GROUP BY username
                                         HAVING COUNT(rating_id) > 48)
                       GROUP BY username)
            
            SELECT AVG(r.count_reviews) AS avg_count_reviews
            FROM r
            
        '''
average_amount_of_reviews = pd.io.sql.read_sql(query, con = engine)
average_amount_of_reviews.head(1)

,avg_count_reviews
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок- 24.

**Общий вывод:**
База данных состоит из пяти таблиц, в которых не было обнаружено дубликатов и пропусков.

При выполнении заданий было выяснено следующее:
- После 1 января 2000 года вышло 819 книг;
- Лидер по количеству обзоров "Сумерки", но оценка при этом средняя;
- Больше всего книг толще 50 страниц у издательств Penguin Books, Vintage и Grand Central Publishing;
- Лидеры по оценке и количеству оценок и средней оценке соавторы книг о Гарри Поттере Дж. К. Роулинг и Мари ГранПре;
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок- 24.